In [16]:
using LinearAlgebra
using SparseArrays
using Plots

In [17]:
# Arnoldi process
function arnoldi(A, u, m)
    n = size(A, 1)
    Q = zeros(n, m+1)
    H = zeros(m+1, m)
    Q[:, 1] = u / norm(u)
    for j in 1:m
        v = A*Q[:, j]
        for i in 1:j
            H[i, j] = Q[:, i]' * v
            v -= H[i, j] * Q[:, i]
        end
        H[j+1, j] = norm(v)
        Q[:, j+1] = v / H[j+1, j]
    end
    Q, H
end

arnoldi (generic function with 1 method)

In [18]:
n = 1000
M = 300
L = 20
#^^^^^^^^^^^^^^^^^^^^^^^#
#  A*X_σ - X_σ * D = B  #
#_______________________#
# LHS
#   A
λ = [i+10 for i in 1:n]
a = randn(n, n)
A = UpperTriangular(a) - diagm(diag(a)) + diagm(λ)
#   Shift
σ = rand(1:9, L)
D = diagm(σ) 
# RHS
B = randn(n, L);

In [19]:
# Initialize
X_σ     = zeros(n, L) 
X_exact = zeros(n, L)
Res     = zeros(M, L);

In [20]:
for l=1:L
    lhs = A - D[l, l]*I
    b = B[:, l]
    X_exact[:, l] = lhs \ b
    Q, H = arnoldi(lhs, b, M)
    for m in 1:M
        s = vcat(norm(b), zeros(m, 1))
        z = H[1:m+1, 1:m] \ s
        x = Q[:, 1:m] * z
        Res[m, l] = norm(b - lhs*x)
        X_σ[:, l] = x
    end
end
norm(X_exact - X_σ)

5.059904458542247e-14